In [1]:
import phoebe as pb
from phoebe import u
import pandas as pd

In [2]:
logger = pb.logger()
# b = pb.Bundle.open('1.bundle')
b = pb.default_binary()

In [3]:
df = pd.read_csv("./TIC_465088681_sec12.csv", sep="\t")
# df = pd.read_csv("./TIC_465088681_200.csv", sep="\t")
watts_units = u.W / (u.m * u.m)
flux =df['flux']
time = df['time']
sigma = df['flux_err']
b.add_dataset('lc', fluxes=flux.values*watts_units, times=time.values*u.day, sigmas = sigma.values*watts_units, overwrite=True)

<ParameterSet: 80 parameters | contexts: compute, figure, constraint, dataset>

In [4]:
b.add_compute('ellc', compute='fastcompute', overwrite=True)
b.add_solver('sampler.emcee',
             compute='fastcompute',
             solver='emcee_solver', overwrite=True)

<ParameterSet: 17 parameters | qualifiers: continue_from_iter, priors, nwalkers, nlags_factor, use_server, continue_from, progress_every_niters, comments, expose_failed, niters, compute, init_from_combine, init_from, init_from_requires, burnin_factor, thin_factor, priors_combine>

In [5]:
print(b.get_parameter("requiv@primary"))
# b.flip_constraint("requiv@primary", solve_for="requivratio@binary")
# b.flip_constraint("requiv@secondary", solve_for="requivsumfrac@binary")
b.flip_constraint("mass@primary", solve_for="sma")
b.flip_constraint("requivratio@binary", solve_for="requiv@primary")
b.flip_constraint("requivsumfrac@binary", solve_for="requiv@secondary")

Parameter: requiv@primary@component
                       Qualifier: requiv
                     Description: Equivalent radius
                           Value: 1.0 solRad
                  Constrained by: 
                      Constrains: logg@primary@component, requivratio@binary@component, requivsumfrac@binary@component
                      Related to: mass@primary@component, logg@primary@component, requiv@secondary@component, requivratio@binary@component, sma@binary@component, requivsumfrac@binary@component



<ConstraintParameter: {requiv@secondary@component} = (({requivratio@binary@component} * {requivsumfrac@binary@component}) * {sma@binary@component}) / ({requivratio@binary@component} + 1.000000) (solar units) => 0.9999999999999999 solRad>

In [6]:
#atmosphere
b.set_value_all("ld_mode", "manual")
b.set_value_all("ld_mode_bol", "manual")
b.set_value_all("atm", "ck2004")
# b.set_value('pblum_mode', 'dataset-scaled')
b.set_value('pblum_mode', 'component-coupled')

b["mass@primary@star"] = 10.4
# b["logg@primary@star@component"] = 4
# b["requiv@primary"] = 8.8

In [11]:
b.add_distribution({
#     'teffratio': pb.gaussian_around(0.1),
                    "requivratio@binary": pb.gaussian_around(1),
#                     'requiv@primary': pb.gaussian_around(0.1),
#                     'incl@binary': pb.gaussian_around(3),
#                     'sma@binary': pb.gaussian_around(2),
                    'q': pb.gaussian_around(0.1)},
#                     'ecc': pb.gaussian_around(0.5),
#                     'per0': pb.gaussian_around(1)},
                    distribution='ball_around_guess', overwrite_all=True)

b.set_value('init_from', 'ball_around_guess') 

In [12]:
## priors
b.add_distribution({"ecc@binary": pb.gaussian(0.26, 0.01),
                    "q@binary": pb.gaussian(5.77, 0.1),
                   "requiv@primary": pb.gaussian(8.8,1.2),
                   "logg@primary": pb.gaussian(4.0,0.5)},
                   distribution='priors_from_external_source', overwrite_all=True)


b.set_value('priors@emcee_solver', 'priors_from_external_source')

In [13]:
print(b.run_checks())
# requivprimary for requiv something else at binary
# requivsecondary for requiv sum frac

Run Checks Report: PASS



In [14]:
# set things first

b.set_value('nwalkers', solver='emcee_solver', value=12)
b.set_value('niters', solver='emcee_solver', value=250)

b.run_solver('emcee_solver', solution='emcee_sol')

/Users/skbowes/anaconda3/lib/python3.10/site-packages/phoebe/dependencies/distl/distl.py:4728: RuntimeWarning: invalid value encountered in divide
  pdf /= pdf_integral
  0%|                                                                                               | 0/12 [00:00<?, ?it/s]

NameError: Constraint 'asini@primary@star@constraint' raised the following error while attempting to solve for 'asini@primary@star@component'.  Consider flipping the constraint or changing the value of one of ['sma@binary@orbit@component', 'incl@binary@orbit@component', 'q@binary@orbit@component', 'asini@primary@star@component'] until the constraint succeeds.  Original error: name 'nan' is not defined

In [15]:
b.plot(solution='emcee_sol', style='lnprobability', show=True)

ValueError: Nothing could be found to plot.  Check all arguments.

In [11]:
b.plot(solution='emcee_sol', style='corner', show=True)

# For full PHOEBE bundle

In [17]:
b.add_distribution({
#     'teffratio': pb.gaussian_around(0.1),
#                     'requivsumfrac': pb.gaussian_around(0.1),
                    'incl@binary': pb.gaussian_around(3),
                    'sma@binary': pb.gaussian_around(2),
                    'q': pb.gaussian_around(0.1)},
#                     'ecc': pb.gaussian_around(0.5),
#                     'per0': pb.gaussian_around(1)},
                    distribution='ball_around_optimized_solution', overwrite_all=True)

b.set_value('init_from', 'ball_around_optimized_solution') 

In [18]:
## priors
b.add_distribution({"ecc@binary": pb.gaussian(0.26, 0.01),
                    "q@binary": pb.gaussian(5.77, 0.1)},
                   distribution='priors_from_external_source', overwrite_all=True)


b.set_value('priors@emcee_solver', 'priors_from_external_source')

In [19]:
# set things first

b.set_value('nwalkers', solver='emcee_solver', value=12)
b.set_value('niters', solver='emcee_solver', value=250)

b.run_solver('emcee_solver', solution='emcee_sol')

Thu, 03 Aug 2023 15:47 BUNDLE       WARNING sampling with dataset-scaled can cause unintended issues.  Consider using component-coupled and marginalizing over pblum
/Users/skbowes/anaconda3/lib/python3.10/site-packages/phoebe/dependencies/distl/distl.py:4728: RuntimeWarning: invalid value encountered in divide
  pdf /= pdf_integral
  0%|                                                                                               | 0/12 [00:00<?, ?it/s]

NameError: Constraint 'logg@primary@star@constraint' raised the following error while attempting to solve for 'logg@primary@star@component'.  Consider flipping the constraint or changing the value of one of ['mass@primary@star@component', 'requiv@primary@star@component', 'logg@primary@star@component'] until the constraint succeeds.  Original error: name 'nan' is not defined

In [37]:
print(b["logg"])

ParameterSet: 4 parameters
C          logg@primary@component: 4.085369359458824
C        logg@secondary@component: 4.437551877570185
          logg@primary@constraint: log10((({mass@primary@component} / ({requiv@primary@component} ** 2.000000)) * 2942.206218) * 9.319541)
        logg@secondary@constraint: log10((({mass@secondary@component} / ({requiv@secondary@component} ** 2.000000)) * 2942.206218) * 9.319541)
